<a href="https://colab.research.google.com/github/Anjasfedo/Learning-TensorFlow/blob/main/eat_tensorflow2_in_30_days/Chapter6_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6-3 Model Training Using Single GPU

The training procedure of deep learning is usually time consuming. It even takes tens of days for training, and there is no need to mention those take days or hours.

The time is mainly consumpted by two stages, data preparation and parameter iteration.

We can increase the number of process to alleviate this issue if data preparation takes the majority of time.

However if the majority of time is taken by parameter iteration, we need to use GPU or Google TPU for acceleration.

There is no need to modify source code for switching from CPU to GPU when using the pre-defined `fit` method or the customized training loops. When GPU is available and the device is not specified, TensorFlow automatically chooses GPU for tensor creating and computing.

However, for the case of using shared GPU with multiple users, such as using server of the company or the lab, we need to add following code to specify the GPU ID and the GPU memory size that we are going to use, in order to avoid the GPU resources to be occupied by a single user (actually TensorFlow aqcuires all GPU cors and all GPU memories by default) and allows multiple users perform training on it.

In Colab notebook, choose GPU in Edit -> Notebook Settings -> Hardware Accelerator

Note: the following code only executes on Colab

In [14]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
2.17.0


In [16]:
from tensorflow.keras import *

# Time stamp
@tf.function
def printbar():
  ts = tf.timestamp()
  today_ts = ts%(24*60*60)

  hour = tf.cast(today_ts%3600, tf.int32)%tf.constant(24)
  minute = tf.cast((today_ts%3600)//60, tf.int32)
  second = tf.cast(tf.floor(today_ts%60), tf.int32)

  def timeformat(m):
    if tf.strings.length(tf.strings.format("{}", m))==1:
      return(tf.strings.format("0{}", m))
    else:
      return(tf.strings.format("{}", m))

  timestrings = tf.strings.join([timeformat(hour), timeformat(minute), timeformat(second)], separator = ":")
  tf.print("=========="*8 + "\n" + timestrings, "\n" + "=========="*8 + "\n")

# 1. GPU Configuration

In [17]:
gpus = tf.config.list_physical_devices("GPU")

if gpus:
    gpu0 = gpus[0] # Only use GPU 0 when existing multiple GPUs
    tf.config.experimental.set_memory_growth(gpu0, True) # Set the usage of GPU memory according to needs
    # The GPU memory usage could also be fixed (e.g. 4GB)
    #tf.config.experimental.set_virtual_device_configuration(gpu0,
    #    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])
    tf.config.set_visible_devices([gpu0],"GPU")

Compare the computing speed between GPU and CPU.

In [3]:
printbar()
with tf.device('/gpu:0'):
  tf.random.set_seed(0)
  a = tf.random.uniform((100000, 100), minval=0, maxval=3.0)
  b = tf.random.uniform((100, 1000), minval=0, maxval=3.0)
  c = a@b

  tf.print(tf.reduce_sum(tf.reduce_sum(c, axis=1), axis=0))
printbar()

In [19]:
printbar()
with tf.device("/cpu:0"):
    tf.random.set_seed(0)
    a = tf.random.uniform((10000,100),minval = 0,maxval = 3.0)
    b = tf.random.uniform((100,100000),minval = 0,maxval = 3.0)
    c = a@b
    tf.print(tf.reduce_sum(tf.reduce_sum(c,axis = 0),axis=0))
printbar()

"================================================================================\n07:38:31" 

2.24953778e+11
"================================================================================\n15:38:39" 



# 2. Data Preparation

In [20]:
MAX_LEN = 300
BATCH_SIZE = 32

(x_train, y_train), (x_test, y_test) = datasets.reuters.load_data()
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=MAX_LEN)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=MAX_LEN)

MAX_WORDS = x_train.max()+1
CAT_NUM = y_train.max()+1

2110848/2110848 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [21]:
ds_train = tf.data.Dataset.from_tensor_slices((x_train, y_train)) \
          .shuffle(buffer_size=1000).batch(BATCH_SIZE) \
          .prefetch(tf.data.experimental.AUTOTUNE).cache()

ds_test = tf.data.Dataset.from_tensor_slices((x_test, y_test)) \
          .shuffle(buffer_size=1000).batch(BATCH_SIZE) \
          .prefetch(tf.data.experimental.AUTOTUNE).cache()

# 3. Model Defining

In [22]:
tf.keras.backend.clear_session()

def create_model():
  model = models.Sequential()

  model.add(layers.Embedding(MAX_WORDS, 7, input_length=MAX_LEN))
  model.add(layers.Conv1D(filters=64, kernel_size=5, activation='relu'))
  model.add(layers.MaxPool1D(2))
  model.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu'))
  model.add(layers.MaxPool1D(2))
  model.add(layers.Flatten())
  model.add(layers.Dense(CAT_NUM, activation='softmax'))
  return(model)

model = create_model()
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d (Conv1D)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

# 4. Model Training

In [23]:
optimizer = optimizers.Nadam()
loss_func = losses.SparseCategoricalCrossentropy()

train_loss = metrics.Mean(name='train_loss')
train_metric = metrics.SparseCategoricalAccuracy(name='train_accuracy')

valid_loss = metrics.Mean(name='valid_loss')
valid_metric = metrics.SparseCategoricalAccuracy(name='valid_accuracy')

In [24]:
@tf.function
def train_step(model, features, labels):
  with tf.GradientTape() as tape:
    predictions = model(features, training=True)
    loss = loss_func(labels, predictions)

  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss.update_state(loss)
  train_metric.update_state(labels, predictions)

In [25]:
@tf.function
def valid_step(model, features, labels):
  predictions = model(features)
  batch_loss = loss_func(labels, predictions)
  valid_loss.update_state(batch_loss)
  valid_metric.update_state(labels, predictions)

In [26]:
def train_model(model, ds_train, ds_valid, epochs):
  for epoch in tf.range(1, epochs+1):
    for features, labels in ds_train:
      train_step(model, features, labels)

    for features, labels in ds_valid:
      valid_step(model, features, labels)

    logs = 'Epoch={}, Loss:{}, Accuracy:{}, Valid Loss:{}, Valid Accuracy:{}'

    if epoch%1 == 0:
      printbar()
      tf.print(tf.strings.format(logs,
        (epoch, train_loss.result(), train_metric.result(), valid_loss.result(), valid_metric.result())))
      tf.print("")

    train_loss.reset_states()
    valid_loss.reset_states()
    train_metric.reset_states()
    valid_metric.reset_states()

In [ ]:
train_model(model, ds_train, ds_test, epochs=6)